<table width="100%" border="3">
  <tbody>
    <tr>
      <td ><img src="https://aidea-web.tw/images/web/logo_white.png" alt="Aidea" width="400"/></td>
      <td align='left'><h1>Exercise 2: Simple Hotspot Prediction algorithm</h1></td>
    </tr>
  </tbody>
</table>
Taxi Hotspot Prediction

# AU AI Summer Programs 2020
* This tutorial uses the Taxi Hotspot Prediction dataset of the AIdea platform.
* Introduce how to write deep learning programs to predict the ride demand of a specific block in a certain period of time.
* This notebook program can be executed in the cloud using Google Colab or Jupyter on a personal computer.

AUAI Summer Taxi Hotspot Prediction Project
https://aidea-web.tw/topic/e26fcd72-9da1-4baa-a6a5-36a326f61072

Pro.Hsueh-Ting Chu EMAIL: htchu.taiwan@gmail.com  FB: https://www.facebook.com/htchu.taiwan

# Exercise 2: Simple solution for Taxi Hotspot Prediction 
* AIdea Taxi Hotspot Prediction project

# (A) AIdea dataset


## Step 1: Load the AIdea Taxi Hotspot Prediction dataset from google drive

In [ ]:
from google_drive_downloader import GoogleDriveDownloader
GoogleDriveDownloader.download_file_from_google_drive(file_id='1ycB6gg5iRyPpMmq9gJ2A4N0dNcQ30pFb',dest_path='./content', unzip=True)

## Step 2: Reads training data of taxi points

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df_train = pd.read_csv('train_hire_stats.csv')
df_train.shape

In [ ]:
df_train.head()

# (B) Check weekdays
0 == Monday
1 == Tuesday
2 == Wednesday
3 == Thursday
4 == Friday
5 == Saturday
6 == Sunday

## Step 3: Convert date string

In [ ]:
#Declare weekday & workday
isworkday = np.ones((len(df_train),), dtype=int)
weekday = np.ones((len(df_train),), dtype=int)

In [ ]:
from datetime import datetime
for index, row in df_train.iterrows():
    dd=datetime.strptime(row['Date'], "%Y-%m-%d")
    weekday[index]= dd.weekday() 
    if weekday[index] >=5 :
        isworkday[index] = 0

## Step 4: Build a new dataframe from the training data

In [ ]:
#Build a new dataframe from the training data
RawX = pd.DataFrame(df_train[["Zone_ID", "Hour_slot"]])
RawX['isworkday'] = isworkday
RawX['weekday'] = weekday
RawX.head()

# (C) # One-Hot-Encoder

## Step 5: OneHotEncoder

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')

## Step 6: fit_transform

In [ ]:
enc.fit_transform(RawX)
enc.categories_

## Step 7: transform

In [ ]:
#Transform data into one hot vector
X = enc.transform(RawX).toarray()
X.shape

In [ ]:
#See the cooked input data
X[0:3, :]

# (D) Training a DNN model for prediction

## Step 8: Resize y into (0,1)

In [ ]:
#Resize y into (0,1)
y = df_train["Hire_count"].values/339.0
y.shape

## Step 9: Define a DNN

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Activation, Dropout

In [ ]:
# create and fit a Baseline Neural Network Model
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=58))
model.add(Dense(units=10, activation='softmax'))
model.add(Dropout(0.1))
model.add(Dense(1))

## Step 10: Compile and Train the model

In [ ]:
model.compile(loss='mse', optimizer='adam')
model.fit(X, y, epochs=20, batch_size=1024, verbose=1)

# (E) Do the prediction

## Step 11: Read the test dataset

In [ ]:
df_test = pd.read_csv('test_hire_stats.csv')
df_test.shape

## Step 12:Convert to weekday

In [ ]:
#Declare weekday & workday
isworkday2 = np.ones((len(df_test),), dtype=int)
weekday2 = np.ones((len(df_test),), dtype=int)

In [ ]:
for index, row in df_test.iterrows():
    dd=datetime.strptime(row['Date'], "%Y/%m/%d")
    weekday2[index]= dd.weekday() 
    if weekday2[index] >=5:
        isworkday2[index] = 0

## Step 13: One-hot encoding of test data

In [ ]:
Test = pd.DataFrame(df_test[["Zone_ID", "Hour_slot"]])
Test['isworkday'] = isworkday2
Test['weekday'] = weekday2

In [ ]:
Xtest = enc.transform(Test).toarray()
Xtest.shape

## Step 14: Do prediction

In [ ]:
yt = model.predict(Xtest)*339.0

## Step 15: Output result

In [ ]:
plt.plot(yt)

In [ ]:
dfOut = pd.DataFrame(df_test)

In [ ]:
for i in range(len(yt)):
  if yt[i] < 0.0:
    yt[i] = 0.0

In [ ]:
dfOut['Hire_count'] = yt

In [ ]:
dfOut.head()

In [ ]:
dfOut.to_csv('Result-A.csv', index=False)